In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import scipy.stats
%matplotlib inline

In [2]:
def undistort(img, nx=9, ny=6):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    op = []
    ip = []
    if ret:
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        
        ip.append(corners)
        objp = np.zeros((nx * ny, 3), np.float32)
        objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
        op.append(objp)
        
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(op, ip, gray.shape[::-1], None, None)
        undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def undistortAll(nx=9, ny=6, debug=False):
    fnames = glob.glob('camera_cal/calibration*.jpg')
    objpoints = []
    imgpoints = []
    shape = []
    
    objp = np.zeros((nx * ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    
    for fname in fnames:
        img = plt.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        shape = gray.shape[::-1]
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        if ret:
            cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            #plt.figure()
            #plt.imshow(img)
            imgpoints.append(corners)
            objpoints.append(objp)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)
    
    if debug:
        for fname in fnames:
            img = plt.imread(fname)
            dst = cv2.undistort(img, mtx, dist, None, mtx)
            plt.figure(figsize=(8,4))
            plt.subplot(2,2,1)
            plt.imshow(img)
            plt.subplot(2,2,2)
            plt.imshow(dst)
            
    return mtx, dist

def getMat():
    src = np.float32([[284,672], [603,444], [678,444 ], [1035,675]])
    dst = np.float32([[400,720], [400,0], [880,0], [880,720]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

In [4]:
mtx, dist = undistortAll(debug=False)
M, Minv = getMat()

In [61]:
def thres(img, s_thresh=(170, 255), sx_thresh=(80, 130), h_thresh=(0,180)):
    img = np.copy(img)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    hcombined_binary = np.zeros_like(sxbinary)
    hcombined_binary[(h_binary == 1) & ((s_binary == 1) | (sxbinary == 1))] = 1

    return hcombined_binary

def transform(img, M):
    warped = cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return warped

def getWarped(img, mtx, dist, M):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    binary = thres(undist)
    warped = transform(binary, M)
    return warped

def weightedHist(hist):
    xlen = hist.shape[0]
    midx = int(xlen / 2)
    guess_left = 400
    guess_right = xlen - guess_left
    guess_width = guess_left / 2
    normx = range(xlen)
    norm_left = scipy.stats.norm.pdf(normx, guess_left, guess_width)
    norm_left /= np.max(norm_left)
    norm_left[midx:] = 1
    norm_right = scipy.stats.norm.pdf(normx, guess_right, guess_width)
    norm_right /= np.max(norm_right)
    norm_right[:midx] = 1
    h2 = hist * norm_right * norm_left
    return h2

def getCurves(lefty, leftx, righty, rightx, y_eval):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    #left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    #right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    #left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    #right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #return left_curverad, right_curverad
    left_curverad = getCurve(lefty, leftx, y_eval)
    right_curverad = getCurve(righty, rightx, y_eval)
    return left_curverad, right_curverad

def getCurve(y, x, y_eval):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(y*ym_per_pix, x*xm_per_pix, 2)
    # Calculate the new radii of curvature
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    # Now our radius of curvature is in meters
    return curverad

In [62]:
def boxSearch(warped, doPlot=False):
    midx = int(warped.shape[1]/2)
    midy = int(warped.shape[0]/2)
    
    # Take a histogram of the bottom half of the image
    histogram0 = np.sum(warped[midy:,:], axis=0)
    histogram = weightedHist(histogram0)

    # Create an output image to draw on and visualize the result
    out_img = np.dstack((warped, warped, warped))*255
    
    # Find the peak of the left and right halves of the histogram
    leftx_base = np.argmax(histogram[:midx])
    rightx_base = np.argmax(histogram[midx:]) + midx

    # Choose the number of sliding windows
    nwindows = 9
    
    # Set height of windows
    window_height = np.int(warped.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    if doPlot:
        plt.figure(figsize=(8,3))
        plt.subplot(1,2,1)
        plt.plot(histogram0)

        plt.subplot(1,2,2)
        plt.plot(histogram)
        plt.show()

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped.shape[0] - (window+1)*window_height
        win_y_high = warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Darw the wdindows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
            (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
            (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If uou found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current  = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    left_curverad, right_curverad = getCurves(lefty, leftx, righty, rightx, warped.shape[1])

    if doPlot:
        # Generate x and y values for plotting
        ploty = np.linspace(0, out_img.shape[0] - 1, out_img.shape[0])
        left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        plt.figure(figsize=(12,8))
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, 'yo')
        plt.plot(right_fitx, ploty, 'yo')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)

    return left_fit, right_fit, left_curverad, right_curverad

In [63]:
def polySearch(warped, left_fit, right_fit, doPlot=False):
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = (
    (nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
    (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = (
    (nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin))&
    (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    left_curverad, right_curverad = getCurves(lefty, leftx, righty, rightx, warped.shape[1])
    
    if doPlot:
        # Generate x and y values for plotting
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((warped, warped, warped))*255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin,ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0, 255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        plt.imshow(result)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        
    return left_fit, right_fit, left_curverad, right_curverad

In [64]:
def plotLines(img, warped, mtx, dist, left_fit, right_fit):
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    undist = cv2.undistort(img, mtx, dist, None, mtx)

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    return result, ploty, left_fitx, right_fitx

In [65]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [82]:
class Line:
    w = 0.1
    def __init__(self, fit, curverad):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        #self.current_fit = [np.array([False])]
        self.fit = fit
        #radius of curvature of the line in some units
        self.curverad = curverad
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.fitx = None  
        #y values for detected line pixels
        self.ploty = None
        
    def addNewFit(self, fit, curverad):
        self.fit = fit
        self.curverad = curverad
        
    def isGoodCurve(self):
        return self.curverad > 200
        
    def isGoodFit(self):
        return self.isGoodCurve()
    
    def updateFit(self):
        if self.isGoodFit():
            if self.best_fit is None:
                self.best_fit = self.fit
            else:
                self.best_fit = self.w*self.fit + (1-self.w)*self.best_fit

class ProcImg:
    leftLine = None
    rightLine = None
    warped = None
    def doFirstSearch(self):
        left_fit, right_fit, left_curverad, right_curverad = \
            boxSearch(self.warped, doPlot=False)
        self.leftLine = Line(left_fit, left_curverad)
        self.rightLine = Line(right_fit, right_curverad)
    def doBoxSearch(self):
        left_fit, right_fit, left_curverad, right_curverad = \
            boxSearch(self.warped, doPlot=False)
        self.leftLine.addNewFit(left_fit, left_curverad)
        self.rightLine.addNewFit(right_fit, right_curverad)
    def doPolySearch(self):
        left_fit, right_fit, left_curverad, right_curverad = \
            polySearch(self.warped, self.leftLine.fit, self.rightLine.fit, doPlot=False)
        self.leftLine.addNewFit(left_fit, left_curverad)
        self.rightLine.addNewFit(right_fit, right_curverad)
        
    def __call__(self, img):
        self.warped = getWarped(img, mtx, dist, M)
    
        if self.leftLine is None:
            self.doFirstSearch()
        else:
            self.doPolySearch()
            if not (self.leftLine.isGoodFit() and self.rightLine.isGoodFit()):
                self.doBoxSearch()
                
        self.leftLine.updateFit()
        self.rightLine.updateFit()

        result, ploty, left_fitx, right_fitx = \
            plotLines(img, self.warped, mtx, dist, self.leftLine.fit, self.rightLine.fit)
        return result

In [ ]:
pimg = ProcImg()
project_output = 'output_images/project.mp4'
clip1 = VideoFileClip("project_video.mp4").subclip(40,42)
project_clip = clip1.fl_image(pimg)
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video output_images/project.mp4
[MoviePy] Writing video output_images/project.mp4


 35%|███▌      | 18/51 [00:04<00:07,  4.23it/s]

In [88]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [14]:
pimg = ProcImg()
chall_output = 'output_images/challenge.mp4'
clip1 = VideoFileClip("challenge_video.mp4").subclip(0, 0.4)
chall_clip = clip1.fl_image(pimg)
%time chall_clip.write_videofile(chall_output, audio=False)

[MoviePy] >>>> Building video output_images/challenge.mp4
[MoviePy] Writing video output_images/challenge.mp4


100%|██████████| 12/12 [00:04<00:00,  3.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/challenge.mp4 

CPU times: user 4.45 s, sys: 584 ms, total: 5.03 s
Wall time: 4.97 s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(chall_output))

In [16]:
pimg = ProcImg()
hchall_output = 'output_images/harder_challenge.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4").subclip(0, 0.4)
hchall_clip = clip1.fl_image(pimg)
%time hchall_clip.write_videofile(hchall_output, audio=False)

[MoviePy] >>>> Building video output_images/harder_challenge.mp4
[MoviePy] Writing video output_images/harder_challenge.mp4


  0%|          | 0/11 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(hchall_output))